<a href="https://colab.research.google.com/github/amandafbri/NLP/blob/master/FuzzyWuzzy_strings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FuzzyWuzzy**

Esse pacote faz match entre strings de maneira "difusa", ou seja, verifica a semelhança entre elas. Utiliza Distância de Levenshtein para calcular a diferença entre as sequências.

### **Setup**

In [21]:
!pip install fuzzywuzzy

In [22]:
from fuzzywuzzy import fuzz,t process

### **Principais recursos**

**Ratio** ⟶ Compara a similaridade da string toda, em ordem.

In [23]:
fuzz.ratio('Hipertensão arterial', 'Hiper tensão arterial')

98

Curiosamente, a pontuação é algo importante no cálculo.

In [24]:
fuzz.ratio('HAS', 'H.A.S.')

67

**Partial Ratio** ⟶ Compara parcialmente a similaridade da string.

In [25]:
fuzz.partial_ratio('Hipertensão arterial', 'Hiper tensão arterial')

95

**Token Sort Ratio** ⟶ Ignora a ordem da string.

In [26]:
fuzz.token_sort_ratio('Hipertensão arterial', 'Hiper tensão arterial')

97

**Token Set Ratio** ⟶ Ignora palavras duplicadas, sendo um pouco mais flexível que o anterior.

In [27]:
fuzz.token_set_ratio('Hiper tensão tensão arterial', 'Hiper tensão arterial')

100

**Extract** ⟶ Retorna o ou os melhores matches de acordo com uma lista de escolhas. Possui vários parâmetros de configuração para definir o "conceito" do que é melhor e também do retorno em si.

In [28]:
choices = ['Nódulo', 'Tumor', 'Câncer', 'Nodulariforme', 'Cancerígeno', 'Lesão']
process.extract('imagem nodular', choices, limit=2)

[('Nódulo', 72), ('Nodulariforme', 52)]

**Extract Best** ⟶ Retorna uma quantidade de melhores matches acima de um limiar de acordo com uma lista de escolhas.

In [29]:
process.extractBests('imagem nodular', choices, score_cutoff=70)

[('Nódulo', 72)]

**Extract One** ⟶ Retorna o melhor match acima de um limiar de acordo com uma lista de escolhas.

In [30]:
process.extractOne('imagem nodular', choices)

('Nódulo', 72)

**Dedupe** ⟶ Recebe uma lista com strings e retorna a mesma sem termos duplicados. Consideram-se duplicadas as strings com similaridade acima de um limiar.

In [31]:
to_dedupe = choices + ['nodulo', 'cancer']
print(to_dedupe)
process.dedupe(choices)

['Nódulo', 'Tumor', 'Câncer', 'Nodulariforme', 'Cancerígeno', 'Lesão', 'nodulo', 'cancer']


['Nódulo', 'Tumor', 'Câncer', 'Nodulariforme', 'Cancerígeno', 'Lesão']

# **Casos de uso**

## **Testando sinônimos de achados médicos**

Optei por fazer a lista com as palavras já pré-processadas como eu gostaria. Sem acentos nem caracteres especiais.

In [32]:
consolidation = ['consolidacao', 'consolidacoes', 'condensacao', 'condensacoes', 'condencacao', 'condencacoes']

In [33]:
nodule = ['nodulo', 'nodular', 'nodulariforme', 'lesao', 'nodularizado']

In [34]:
mixed_findings = ['cardiomegalia', 'arritmia', 'fibrilacao atrial', 'derrame', 'cancer']

In [35]:
process.extractOne('condensante', consolidation+nodule+mixed_findings)

('condensacoes', 78)

Quando observamos os 5 melhores resultados, vemos o quão delicada é a escolha do limiar. A palavra "lesão" e "condensante" são muito diferentes!

In [36]:
process.extractBests('condensante', consolidation+nodule+mixed_findings)

[('condensacoes', 78),
 ('condensacao', 73),
 ('condencacao', 64),
 ('condencacoes', 61),
 ('lesao', 54)]

In [37]:
process.extractOne('noduliforme', consolidation+nodule+mixed_findings)

('nodulariforme', 92)

Mais um caso com palavras bem distintas. Se buscamos por sinônimos, o limiar deve ser bem alto para garantir coerência.

In [38]:
process.extractBests('noduliforme', consolidation+nodule+mixed_findings)

[('nodulariforme', 92),
 ('nodulo', 75),
 ('nodular', 67),
 ('nodularizado', 61),
 ('consolidacoes', 50)]

Vamos ver como a função *dedupe* se comporta. Removeu apenas "nodulo". Quem sabe com algunas ajustes de limiar e qual tipo de *scorer* (aqueles que testamos com *fuzz.(...)*)

In [39]:
print(nodule)
process.dedupe(nodule)

['nodulo', 'nodular', 'nodulariforme', 'lesao', 'nodularizado']


dict_keys(['nodular', 'nodularizado', 'nodulariforme', 'lesao'])

Com *partial_ratio* já obtivemos uma melhoria!

In [40]:
print(nodule)
process.dedupe(nodule, scorer=fuzz.partial_ratio)

['nodulo', 'nodular', 'nodulariforme', 'lesao', 'nodularizado']


dict_keys(['nodulariforme', 'lesao'])

## **Testando presença de achados médicos**

Um problema muito comum em processamento de linguagem natural refere-se à afirmação ou negação de uma informação em determinados textos. 

No contexto de saúde, isso também ocorre. Em laudos médicos, por exemplo, podemos extrair informações clínicas muito relevantes (como vimos no caso anterior), mas a parte mais delicada é justamente o estado daquilo que queremos extrair.

Para nódulos, por exemplo, poderíamos ter:

*   "Ausência de nódulos" 
*   "Apresenta nódulo calcificado"
*   "Imagem nodular a esclarecer"

**Todas as frases se referem a nódulos, mas com significados completamente distintos!**

Vamos aos nossos testes então! Primeiro vou definir a lista de escolhas com opções de expressões que representam a negação ou dúvida do achado.

In [41]:
absence = ['ausencia de', 'sem sinais de', 'sem sinal de', 'nao evidenciamos sinais de', 'nao evidenciamos sinal de', 'nao ha evidencias', 'nao ha sinal', 'nao ha sinais']

In [42]:
clue = ['aparente', 'questionavel', 'possivel', 'equivoca', 'a esclarecer']

In [43]:
choices = absence + clue

Agora podemos verificar o comportamento da extração. Nesse primeiro caso, foi fácil... Os *scores* ficaram bem altos.

In [45]:
process.extractBests('nao tem nenhuma evidencia', absence)

[('nao ha sinal', 86),
 ('nao ha sinais', 86),
 ('nao ha evidencias', 76),
 ('nao evidenciamos sinal de', 61),
 ('nao evidenciamos sinais de', 60)]

Já nesse exemplo, os *scores* ficaram muito baixos, apesar de ter acertado as duas expressões que aparecem primeiro.

In [46]:
process.extractBests('parece ser um nodulo', choices)

[('aparente', 56),
 ('a esclarecer', 50),
 ('sem sinal de', 45),
 ('sem sinais de', 41),
 ('ausencia de', 32)]

Mais uma opção que acertou bem a negação, muito provavelmente pela palavra "não" estar presente.

In [47]:
process.extractBests('com certeza absoluta nao e cardiomegalia', choices)

[('nao evidenciamos sinais de', 86),
 ('nao evidenciamos sinal de', 86),
 ('nao ha evidencias', 86),
 ('nao ha sinal', 86),
 ('nao ha sinais', 86)]

Já com uma frase que indica presença, os *scores* ficaram bem baixos, correspondendo às expectativas, porque não temos na lista nada referente ao caso.

In [48]:
process.extractBests('e um nodulo', choices)

[('sem sinal de', 43),
 ('sem sinais de', 42),
 ('nao ha sinal', 35),
 ('nao ha evidencias', 34),
 ('equivoca', 32)]

O exemplo abaixo é um tanto subjetivo. A expressão "deve ser" sugere ser uma pista (*clue*), mas a extração achou mais próximo da negação. Porém, pelos *scores* baixos, ficou muito semelhante ao caso anterior, de afirmação. 

In [49]:
process.extractBests('deve ser um nodulo', choices)

[('sem sinal de', 45),
 ('nao evidenciamos sinal de', 40),
 ('nao evidenciamos sinais de', 39),
 ('sem sinais de', 37),
 ('nao ha sinal', 36)]

Para outro exemplo de afirmação do achado, fiquei bastante surpresa com o resultado. Como as palavras "presente" e "aparente" são semelhantes, o *score* foi bem alto, mas foi a única possibilidade concreta.

In [50]:
process.extractBests('nodulo presente', choices)

[('aparente', 68),
 ('ausencia de', 38),
 ('nao ha evidencias', 38),
 ('nao ha sinal', 37),
 ('nao evidenciamos sinais de', 36)]

Fazendo para as listas de negação e pista separadamente:

No primeiro exemplo, acertou com facilidade pela semelhança das palavras ser forte.

In [51]:
print(process.extractBests('possivelmente um nodulo', clue))
print('--------------')
print(process.extractBests('possivelmente um nodulo', absence))

[('possivel', 90), ('aparente', 45), ('questionavel', 43), ('a esclarecer', 28), ('equivoca', 22)]
--------------
[('sem sinal de', 38), ('sem sinais de', 36), ('nao ha sinal', 36), ('nao ha sinais', 32), ('nao evidenciamos sinais de', 28)]


Já nos exemplos a seguir, temos um caso curioso. Ambas tiveram *scores* semelhantes para a lista *clue*. Para a lista *absence* a diferença ficou um pouco mais coerente.

In [52]:
print(process.extractBests('apresenta um nodulo', clue))
print('--------------')
print(process.extractBests('apresenta um nodulo', absence))

[('aparente', 68), ('questionavel', 36), ('possivel', 34), ('a esclarecer', 26), ('equivoca', 22)]
--------------
[('ausencia de', 50), ('sem sinal de', 45), ('sem sinais de', 38), ('nao ha sinal', 38), ('nao ha evidencias', 37)]


In [54]:
print(process.extractBests('nao apresenta nodulo', clue))
print('--------------')
print(process.extractBests('nao apresenta nodulo', absence))

[('aparente', 68), ('questionavel', 38), ('a esclarecer', 38), ('possivel', 34), ('equivoca', 28)]
--------------
[('nao ha sinal', 86), ('nao ha sinais', 86), ('ausencia de', 50), ('nao evidenciamos sinal de', 49), ('nao ha evidencias', 49)]


Por fim, podemos escolher diferentes parâmetros e ver como eles podem impactar no resultado.

No exemplo abaixo, podemos notar a mega diferença que destaca o resultado quando usamos *partial ratio*. Ou seja, a escolha da configuração é essencial!

In [62]:
print('Ratio')
print(process.extract('nao apresenta nodulo', choices, scorer=fuzz.ratio, limit=2))
print('Partial ratio')
print(process.extract('nao apresenta nodulo', choices, scorer=fuzz.partial_ratio, limit=2))
print('Token set ratio')
print(process.extract('nao apresenta nodulo', choices, scorer=fuzz.token_set_ratio, limit=2))
print('Token sort ratio')
print(process.extract('nao apresenta nodulo', choices, scorer=fuzz.token_sort_ratio, limit=2))

Ratio
[('nao ha sinal', 50), ('nao evidenciamos sinal de', 49)]
Partial ratio
[('aparente', 75), ('nao ha sinais', 62)]
Token set ratio
[('nao ha sinal', 50), ('nao evidenciamos sinal de', 49)]
Token sort ratio
[('nao ha sinal', 50), ('nao evidenciamos sinal de', 49)]


**Referências**


*   https://pypi.org/project/fuzzywuzzy/
*   https://github.com/seatgeek/fuzzywuzzy

